In [1]:
%pip install -r ../requirements.txt


print("[LOG] Requirements installed.")

Note: you may need to restart the kernel to use updated packages.
[LOG] Requirements installed.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os, getpass

load_dotenv()
print("[LOG] .env loaded.")

# Prompt only if missing
for key in ["QDRANT_URL", "QDRANT_API_KEY", "GROQ_API_KEY"]:
    if not os.getenv(key):
        prompt_text = f"{key} not found in .env. Paste it now (won't be saved):"
        os.environ[key] = getpass.getpass(prompt_text)

# Ensure Qdrant and Groq keys exist
if not os.getenv("QDRANT_URL") or not os.getenv("QDRANT_API_KEY"):
    raise RuntimeError("Qdrant keys are required to run this notebook.")
if not os.getenv("GROQ_API_KEY"):
    print("[WARN] Groq key missing. Only local fallback generation will work (not recommended).")

print("[LOG] Keys status -> Qdrant:", bool(os.getenv("QDRANT_URL")), 
      "Groq:", bool(os.getenv("GROQ_API_KEY")))



[LOG] .env loaded.
[LOG] Keys status -> Qdrant: True Groq: True


In [3]:
# Cell-3: Index data to Qdrant Cloud

import os

csv_path = "../data/rag_optimized_5000.csv"

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"CSV file not found at {csv_path}")

# Optional: quick check of CSV
import pandas as pd
df = pd.read_csv(csv_path)
print(f"[LOG] Loaded {len(df)} rows from CSV.")

# Run the indexing script with Qdrant only
!python ../src/load_data.py --csv ../data/rag_optimized_5000.csv

print("[LOG] Indexing script executed. All docs should now be in Qdrant Cloud.")

[LOG] Loaded 5000 rows from CSV.
[LOG] Indexing 5000 document chunks...
[RAG] Indexing 5000 docs with batch_size=64
[Embedder] Embedding 5000 texts with batch_size=64
[Embedder] Batch 1: 64 texts
[Embedder] Batch 2: 64 texts
[Embedder] Batch 3: 64 texts
[Embedder] Batch 4: 64 texts
[Embedder] Batch 5: 64 texts
[Embedder] Batch 6: 64 texts
[Embedder] Batch 7: 64 texts
[Embedder] Batch 8: 64 texts
[Embedder] Batch 9: 64 texts
[Embedder] Batch 10: 64 texts
[Embedder] Batch 11: 64 texts
[Embedder] Batch 12: 64 texts
[Embedder] Batch 13: 64 texts
[Embedder] Batch 14: 64 texts
[Embedder] Batch 15: 64 texts
[Embedder] Batch 16: 64 texts
[Embedder] Batch 17: 64 texts
[Embedder] Batch 18: 64 texts
[Embedder] Batch 19: 64 texts
[Embedder] Batch 20: 64 texts
[Embedder] Batch 21: 64 texts
[Embedder] Batch 22: 64 texts
[Embedder] Batch 23: 64 texts
[Embedder] Batch 24: 64 texts
[Embedder] Batch 25: 64 texts
[Embedder] Batch 26: 64 texts
[Embedder] Batch 27: 64 texts
[Embedder] Batch 28: 64 texts
[E


Embedding batches: 100%|██████████| 79/79 [00:54<00:00,  1.45it/s]


In [4]:
import sys, os
sys.path.append(os.path.abspath("../src"))
print("[LOG] src/ added to sys.path")


[LOG] src/ added to sys.path


In [5]:
# Cell-4: Query RAG with Qdrant + Groq

from rag import RAG

# Initialize RAG: Qdrant + Groq
rag = RAG()

# Example query
query = "How do I reset my password?"
print("[LOG] Query:", query)

# Retrieve relevant docs
retrieved = rag.retrieve(query, top_k=5)
print(f"[LOG] Retrieved {len(retrieved)} docs:")

for i, doc in enumerate(retrieved, 1):
    print(f"Doc {i}:", doc[:120], "...")

# Generate answer using Groq if available
print("\n[LOG] Generating answer...")
answer = rag.answer(query, top_k=5)
print("\n[Answer]:\n", answer)


d:\Naman\Projects\RAG\QuickRAG-FAQ-Bot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[LOG] Query: How do I reset my password?
[RAG] Retrieving top 5 docs for query: How do I reset my password?
[Embedder] Embedding 1 texts with batch_size=64


Embedding batches: 100%|██████████| 1/1 [00:00<00:00, 38.56it/s]

[Embedder] Batch 1: 1 texts
[Embedder] Final embeddings shape: (1, 384)
[QdrantDB] Searching for top 5 docs.


[QdrantDB] Found 0 hits.
[RAG] Retrieved 0 docs.
[LOG] Retrieved 0 docs:

[LOG] Generating answer...
[RAG] Answering query: How do I reset my password?
[RAG] Retrieving top 5 docs for query: How do I reset my password?
[Embedder] Embedding 1 texts with batch_size=64


Embedding batches: 100%|██████████| 1/1 [00:00<00:00, 111.10it/s]

[Embedder] Batch 1: 1 texts
[Embedder] Final embeddings shape: (1, 384)
[QdrantDB] Searching for top 5 docs.


[QdrantDB] Found 0 hits.
[RAG] Retrieved 0 docs.
[RAG] Using Groq for generation.

[Answer]:
 To reset your password, follow these steps:

1. Go to the login page.
2. Click on the "Forgot Password" or "Reset Password" option (Retrieved from [1] - General Password Reset Procedure).
3. Enter your username, email address, or phone number associated with your account (Retrieved from [2] - Account Recovery Process).
4. Follow the instructions provided to reset your password (Retrieved from [3] - Password Reset Guidelines).

Remember to keep your new password secure and unique.


In [6]:
# Cell-5: Quick Qdrant + Groq inference demo

from rag import RAG

# Initialize RAG: Qdrant + Groq
rag = RAG()

# Example queries for quick test
queries = [
    "When is the library open?",
    "How do I reset my password?"
]

for query in queries:
    print("\n[LOG] Query:", query)
    
    # Retrieve top docs from Qdrant
    retrieved = rag.retrieve(query, top_k=5)
    print(f"[LOG] Retrieved {len(retrieved)} docs:")
    for i, doc in enumerate(retrieved, 1):
        print(f"Doc {i}:", doc[:120], "...")
    
    # Generate answer using Groq
    print("\n[LOG] Generating answer...")
    answer = rag.answer(query, top_k=5)
    print("\n[Answer]:\n", answer)



[LOG] Query: When is the library open?
[RAG] Retrieving top 5 docs for query: When is the library open?
[Embedder] Embedding 1 texts with batch_size=64


Embedding batches: 100%|██████████| 1/1 [00:00<00:00, 73.88it/s]

[Embedder] Batch 1: 1 texts
[Embedder] Final embeddings shape: (1, 384)
[QdrantDB] Searching for top 5 docs.


[QdrantDB] Found 0 hits.
[RAG] Retrieved 0 docs.
[LOG] Retrieved 0 docs:

[LOG] Generating answer...
[RAG] Answering query: When is the library open?
[RAG] Retrieving top 5 docs for query: When is the library open?
[Embedder] Embedding 1 texts with batch_size=64


Embedding batches: 100%|██████████| 1/1 [00:00<00:00, 111.09it/s]

...

[Embedder] Batch 1: 1 texts
[Embedder] Final embeddings shape: (1, 384)
[QdrantDB] Searching for top 5 docs.


[QdrantDB] Found 0 hits.
[RAG] Retrieved 0 docs.
[LOG] Retrieved 0 docs:

[LOG] Generating answer...
[RAG] Answering query: How do I reset my password?
[RAG] Retrieving top 5 docs for query: How do I reset my password?
[Embedder] Embedding 1 texts with batch_size=64


Embedding batches: 100%|██████████| 1/1 [00:00<00:00, 111.38it/s]

[Embedder] Batch 1: 1 texts
[Embedder] Final embeddings shape: (1, 384)
[QdrantDB] Searching for top 5 docs.


[QdrantDB] Found 0 hits.
[RAG] Retrieved 0 docs.
[RAG] Using Groq for generation.

[Answer]:
 To reset your password, follow these steps:

1. Go to the login page of the account you need to reset the password for. [Retrieved from "How to Login/Sign Up" - Company Website]
2. Click on the "Forgot Password" or "Reset Password" link. [Retrieved from "Account Recovery" - Company Website]
3. Enter your email address or username associated with the account. [Retrieved from "Account Recovery" - Company Website]
4. Click on the "Send Reset Email" or "Reset Password" button. [Retrieved from "Account Recovery" - Company Website]
5. Check your email for a reset password link or instructions. [Retrieved from "Company Email Policy" - Company Website]
6. Follow the instructions in the email to reset your password.

Please note: the specific steps may vary depending on the platform or website you are using. [Retrieved from "Platform-Specific Instructions" - Online Documentation]

(Company Website: gen